In [2]:
import json
import torch
import json
import os
from LayoutDM import CLDM
from dataset import ImageLayout_Val
from torch.utils.data import Dataset, DataLoader
from diffusers import DDPMScheduler
from PIL import Image
import numpy as np
import gc
from tqdm import tqdm

In [3]:
diffusion = DDPMScheduler(num_train_timesteps= 1000)

In [4]:
device = f"cuda:{6}" if torch.cuda.is_available() else "cpu"

In [5]:
def seg_loss(predicted_box, sample, zero_count):
    src = sample['sr']
    src_list = []
    
    for i in src:
        src_ = Image.open(i)
        src_list.append(src_)
    
    box = predicted_box.cpu().numpy()
    box = (box + 1) / 2
    
    match_list = []

    for i in range(box.shape[0]):
        img = src_list[i]
        width, height = img.size
        cx, cy, w, h = box[i]
        x = int((cx - w / 2) * width)
        y = int((cy - h / 2) * height)
        x2 = int((cx + w / 2) * width)
        y2 = int((cy + h / 2) * height)
        boxes = (x, y, x2, y2)
        
        # 이미지 크롭
        crop = img.crop(boxes)
        crop = np.array(crop)


        if crop.size == 0:
            print(f"Warning: Crop size is zero for box {boxes}.")
            match_list.append(0)  
            zero_count+=1
            continue
        
        blue_channel = crop[:, :, 2]
        blue_channel_flatten = blue_channel.flatten()
        

        match_pixel_size = np.sum(blue_channel_flatten == 128) / blue_channel_flatten.size
        match_list.append(match_pixel_size)
    
    # NaN 방지를 위해 match_list가 비어 있지 않은지 확인
    if len(match_list) == 0:
        print("Error: All crops have zero size. Returning NaN.")
        return float('nan')
    
    value = sum(match_list) / len(match_list)
    print(value)
    return value, zero_count

## Load Model with Seperate Param dict

In [6]:
val_path = "/nas2/lait/1000_Members/jjoonvely/carla_new/pre_seg_combined_val.json" 

In [7]:
val = ImageLayout_Val(val_path)
dataset = DataLoader(val, batch_size=256)

In [8]:
epoch = [100,150,200,250,300]

In [11]:
src={0: {'score': 0.5814610461090901, 'zero_count': 67}}
for index,value in enumerate(epoch):
    save_path = f"/workspace/joonsm/City_Layout/log_dir/baseline_512/checkpoints/checkpoint-{value}/pytorch_model.bin"
    model = CLDM(use_temp=False)
    model.load_state_dict(torch.load(save_path, map_location=device))
    model = model.to(device)
    model.eval()
    zero_count = 0
    batch_value = []
    step = 0
    with torch.no_grad():
        for step,batch in tqdm(enumerate(dataset), total=len(dataset)):
            shape = batch['box'].shape
            noisy_batch = {'image':batch['image'].to(device),
                    'box': torch.rand(*shape, dtype=torch.float32, device=device)}
            for i in range(250)[::-1]:
                t = torch.tensor([i]*shape[0], device=device)
                with torch.no_grad():
                    noise_pred = model(noisy_batch, timesteps=t)
                    bbox_pred = diffusion.step(noise_pred, t[0].detach().item(),  noisy_batch['box'], return_dict=True)
                    noisy_batch['box'] = bbox_pred.prev_sample
            predicted = bbox_pred.prev_sample
            value,zero_count = seg_loss(predicted, batch,zero_count)
            print(zero_count)
            batch_value.append(value)
            step +=1 

        final = sum(batch_value)/step
        src[index]={'score': final, 'zero_count':zero_count}
    del model
    gc.collect()


  0%|          | 0/9 [00:00<?, ?it/s]

 11%|█         | 1/9 [03:53<31:11, 233.90s/it]

0.6671114893710176
3


 22%|██▏       | 2/9 [07:16<25:08, 215.44s/it]

0.6979320155629881
3


...

0.8907291202407682
0


 89%|████████▉ | 8/9 [30:19<03:47, 227.00s/it]

0.7485442818691056
0


100%|██████████| 9/9 [32:33<00:00, 217.05s/it]

0.675669929832084
0


In [12]:
src

{0: {'score': 0.6458715720668938, 'zero_count': 10},
 1: {'score': 0.77192767936356, 'zero_count': 6},
 2: {'score': 0.767228039949551, 'zero_count': 0},
 3: {'score': 0.7398267173943907, 'zero_count': 0},
 4: {'score': 0.8360930199337169, 'zero_count': 0}}